Real data input

In [ ]:
import pandas as pd
import datetime

def process_csv(filepath, product_index):
    trade_data = pd.read_csv(filepath)

    trade_data["TIME_PERIOD"] = pd.to_datetime(trade_data["TIME_PERIOD"], format = '%Y-%m').dt.date
    trade_data = trade_data.rename(columns={"partner": "PARTNER_codes", "declarant": "DECLARANT_codes"})
    trade_data = trade_data.drop_duplicates()
    #trade_data = trade_data.dropna()

    products = trade_data["product"].drop_duplicates().to_list()
    product_filter = products[product_index]

    partner_countries = pd.read_csv("trade_data/partners.csv", sep = ";")

    trade_data = pd.merge(trade_data, partner_countries, on = ['PARTNER_codes'], how = 'inner')

    trade_data_pivot = trade_data.pivot(index=['PARTNER_Labels', 'TIME_PERIOD', 'product'],
                                        columns='indicators',
                                        values=['OBS_VALUE'])

    trade_data_pivot.columns = trade_data_pivot.columns.map('_'.join)
    trade_data_pivot = trade_data_pivot.reset_index()
    trade_data_pivot = trade_data_pivot.dropna()

    df = trade_data_pivot
    df_product_filtered = df[df['product'] == product_filter]

    return df_product_filtered

#Declare DataFrames for all good/productcode combinations
df_sunflower_oil_0 = process_csv("trade_data/sunflower_oil.csv", 0)
df_sunflower_oil_1 = process_csv("trade_data/sunflower_oil.csv", 1)
df_maize_corn_0 = process_csv("trade_data/maize_corn.csv", 0)
df_wheat_0 = process_csv("trade_data/wheat.csv", 0)
df_wheat_1 = process_csv("trade_data/wheat.csv", 1)
df_wheat_2 = process_csv("trade_data/wheat.csv", 2)

Univariate series (test) data input with n-steps

In [1]:
# Import packages
from numpy import array

In [6]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X = list()
    for i in range(len(sequence)):
        #find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x = sequence[i:end_ix]
        X.append(seq_x)
    return array(X)

In [11]:
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 3
# split into samples
X = split_sequence(raw_seq, n_steps)
# summarize the data
print(X)

[[10 20 30]
 [20 30 40]
 [30 40 50]
 [40 50 60]
 [50 60 70]
 [60 70 80]]


In [22]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X = list()
    for i in range(np.shape(sequences)[1]):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > np.shape(sequences)[1]:
            break
        # gather input and output parts of the pattern
        bag_x= sequences[:,i:end_ix]
        X.append(bag_x)
    return array(X)

In [23]:
import numpy as np
# define input sequence
A = np.random.randint(1,10,size = (3,10))
# choose a number of time steps
n_steps = 3
print(A)
# split into samples
X = split_sequences(A, n_steps)
# summarize the data
print(X)

[[8 2 4 2 3 9 3 3 1 9]
 [6 8 7 4 7 5 2 4 9 2]
 [6 5 3 1 3 9 5 6 7 2]]
[[[8 2 4]
  [6 8 7]
  [6 5 3]]

 [[2 4 2]
  [8 7 4]
  [5 3 1]]

 [[4 2 3]
  [7 4 7]
  [3 1 3]]

 [[2 3 9]
  [4 7 5]
  [1 3 9]]

 [[3 9 3]
  [7 5 2]
  [3 9 5]]

 [[9 3 3]
  [5 2 4]
  [9 5 6]]

 [[3 3 1]
  [2 4 9]
  [5 6 7]]

 [[3 1 9]
  [4 9 2]
  [6 7 2]]]
